### Import Packages ###

In [0]:
# Comment code below after running the mlflow update
%pip install "mlflow-skinny[databricks]>=2.4.1"
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
import warnings
warnings.filterwarnings("ignore")
import mlflow
from mlflow.models.signature import infer_signature
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from mlflow import MlflowClient

### Define Catalog and Schema in Unity Catalog ###

In [0]:
catalog = "data_science"
schema = "models"

### Load Kaggle Data Set ###

https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [0]:
data = pd.read_csv('/Workspace/Users/awnish.choudhary@anthology.ai/healthcare-dataset-stroke-data.csv')

### Clean the Data ###

In [0]:
# Convert string columns to numeric values in data
data['gender'] = data['gender'].map({'Male': 0, 'Female': 1, 'Other': 2})
data['ever_married'] = data['ever_married'].map({'No': 0, 'Yes': 1})
data['work_type'] = data['work_type'].map({'Private': 0, 'Self-employed': 1, 'Govt_job': 2, 'children': 3, 'Never_worked': 4})
data['Residence_type'] = data['Residence_type'].map({'Urban': 0, 'Rural': 1})
data['smoking_status'] = data['smoking_status'].map({'Unknown': 0, 'never smoked': 1, 'formerly smoked': 2, 'smokes': 3})

data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,0,67.0,0,1,1,0,0,228.69,36.6,2,1
1,51676,1,61.0,0,0,1,1,1,202.21,NaN,1,1
2,31112,0,80.0,0,1,1,0,1,105.92,32.5,1,1
3,60182,1,49.0,0,0,1,0,0,171.23,34.4,3,1
4,1665,1,79.0,1,0,1,1,1,174.12,24.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,1,80.0,1,0,1,0,0,83.75,NaN,1,0
5106,44873,1,81.0,0,0,1,1,0,125.20,40.0,1,0
5107,19723,1,35.0,0,0,1,1,1,82.99,30.6,1,0
5108,37544,0,51.0,0,0,1,0,1,166.29,25.6,2,0


In [0]:
# Drop rows with missing values
data.dropna(inplace=True)

In [0]:
data = spark.createDataFrame(data)

In [0]:
# Select features and target
featureCols = [col for col in data.columns if col != "stroke"]
assembler = VectorAssembler(inputCols=featureCols, outputCol="features")
data_prepared = assembler.transform(data).select(col("features"), col("stroke").alias("label"))

# Split the data
(train_data, test_data) = data_prepared.randomSplit([0.7, 0.3])

### Model training ###

In [0]:
# Train a RandomForest model
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)
model = rf.fit(train_data)
model_name = 'random_forest_classifier'
# Make predictions
predictions = model.transform(test_data)
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy = {accuracy}")


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Test Accuracy = 0.9591973244147157


### Add model signature which is required to register the model ###

In [0]:
# Sample Data
sample = train_data.limit(5)
# Convert Spark DataFrame to Pandas DataFrame
train_data_pd = sample.toPandas()
# Infer model signature using the Pandas DataFrame
signature = infer_signature(train_data_pd, model.transform(sample).toPandas())

### Log and Register model to MLflow ###

In [0]:
with mlflow.start_run():
    mlflow.spark.log_model(model, model_name, signature=signature)
    uri = mlflow.get_artifact_uri(model_name)
    # Log metrics
    mlflow.log_metric("accuracy_score", accuracy)
    mlflow.set_registry_uri("databricks-uc")
    # Register Model
    mlflow.register_model(
        model_uri=uri,
        name=f"{catalog}.{schema}.{model_name}"
    )

2024/09/17 14:09:06 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


2024/09/17 14:09:08 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2024/09/17 14:09:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/2302940123439732/d153c62e099046188ceb9d7eaf82f303/artifacts/random_forest_classifier/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.1']. Set logging level to DEBUG to see the full traceback. 


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Successfully registered model 'data_science.models.random_forest_classifier'.
2024/09/17 14:09:37 WARNING mlflow.store._unity_catalog.registry.rest_store: Unable to get model version source run's workspace ID from request headers. No run link will be recorded for the model version


2024/09/17 14:09:40 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


Uploading artifacts:   0%|          | 0/24 [00:00<?, ?it/s]

2024/09/17 14:09:41 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Created version '1' of model 'data_science.models.random_forest_classifier'.


### Create "Champion" alias for latest version of the model currently in production  ###

In [0]:
# Initialize the MLflow client
client = MlflowClient()

# Search for all versions of the model and fetch the latest one
model_version_infos = client.search_model_versions(f"name='{catalog}.{schema}.{model_name}'")
new_model_version = max(model_version_info.version for model_version_info in model_version_infos)

# Set the alias for the latest model version
client.set_registered_model_alias(
    name=f"{catalog}.{schema}.{model_name}",
    alias="Champion",
    version=new_model_version
)

### Load Registered Model for inference ###

In [0]:
model_version_uri = 'models:/'+f"{catalog}.{schema}.{model_name}@Champion"
champion_version = mlflow.spark.load_model(model_version_uri)

2024/09/17 14:09:43 INFO mlflow.spark: 'models:/data_science.models.random_forest_classifier@Champion' resolved as 's3://caden-os-prod-databricks-metastore-storage/7adaa701-ad6e-4968-af13-eddf695e53bc/models/a36649dc-65ca-4b5b-9d16-f23d100dc545/versions/e4350f3e-96f7-48cd-a6e1-a8188a955911'


2024/09/17 14:09:46 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


In [0]:
display(champion_version.transform(test_data).limit(5))

features,label,rawPrediction,probability,prediction
"Map(vectorType -> sparse, length -> 11, indices -> List(0, 1, 2, 5, 8, 9), values -> List(15525.0, 1.0, 63.0, 1.0, 96.26, 31.8))",0,"Map(vectorType -> dense, length -> 2, values -> List(96.84831067226501, 3.151689327734992))","Map(vectorType -> dense, length -> 2, values -> List(0.9684831067226501, 0.031516893277349925))",0.0
"Map(vectorType -> sparse, length -> 11, indices -> List(0, 1, 2, 5, 8, 9), values -> List(26134.0, 1.0, 28.0, 1.0, 111.22, 25.5))",0,"Map(vectorType -> dense, length -> 2, values -> List(97.8523946574489, 2.1476053425511097))","Map(vectorType -> dense, length -> 2, values -> List(0.9785239465744889, 0.021476053425511097))",0.0
"Map(vectorType -> sparse, length -> 11, indices -> List(0, 1, 2, 5, 8, 9), values -> List(32257.0, 1.0, 47.0, 1.0, 210.95, 50.1))",0,"Map(vectorType -> dense, length -> 2, values -> List(96.8129161270661, 3.1870838729338957))","Map(vectorType -> dense, length -> 2, values -> List(0.9681291612706611, 0.031870838729338956))",0.0
"Map(vectorType -> sparse, length -> 11, indices -> List(0, 1, 2, 5, 8, 9), values -> List(32689.0, 1.0, 48.0, 1.0, 84.38, 27.1))",0,"Map(vectorType -> dense, length -> 2, values -> List(97.74741060448416, 2.2525893955158467))","Map(vectorType -> dense, length -> 2, values -> List(0.9774741060448416, 0.022525893955158468))",0.0
"Map(vectorType -> sparse, length -> 11, indices -> List(0, 1, 2, 5, 8, 9), values -> List(62783.0, 1.0, 76.0, 1.0, 198.02, 38.7))",0,"Map(vectorType -> dense, length -> 2, values -> List(86.25338464331385, 13.746615356686148))","Map(vectorType -> dense, length -> 2, values -> List(0.8625338464331385, 0.13746615356686148))",0.0
